<a href="https://colab.research.google.com/github/vvicky30/machine-learning/blob/master/desicion_ID3ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import math
import csv
def load_csv(filename):
    lines=csv.reader(open(filename,"r"));
    dataset = list(lines)
    headers = dataset.pop(0)
    return dataset,headers

class Node:
    def __init__(self,attribute):
        self.attribute=attribute
        self.children=[]
        self.answer=""
        
def subtables(data,col,delete):
    dic={}
    coldata=[row[col] for row in data]
    attr=list(set(coldata))
    
    counts=[0]*len(attr)
    r=len(data)
    c=len(data[0])
    for x in range(len(attr)):
        for y in range(r):
            if data[y][col]==attr[x]:
                counts[x]+=1
        
    for x in range(len(attr)):
        dic[attr[x]]=[[0 for i in range(c)] for j in range(counts[x])]
        pos=0
        for y in range(r):
            if data[y][col]==attr[x]:
                if delete:
                    del data[y][col]
                dic[attr[x]][pos]=data[y]
                pos+=1
    return attr,dic
    
def entropy(S):
    attr=list(set(S))
    if len(attr)==1:
        return 0
    
    counts=[0,0]
    for i in range(2):
        counts[i]=sum([1 for x in S if attr[i]==x])/(len(S)*1.0)
    
    sums=0
    for cnt in counts:
        sums+=-1*cnt*math.log(cnt,2)
    return sums

def compute_gain(data,col):
    attr,dic = subtables(data,col,delete=False)
    
    total_size=len(data)
    entropies=[0]*len(attr)
    ratio=[0]*len(attr)
    
    total_entropy=entropy([row[-1] for row in data])
    for x in range(len(attr)):
        ratio[x]=len(dic[attr[x]])/(total_size*1.0)
        entropies[x]=entropy([row[-1] for row in dic[attr[x]]])
        total_entropy-=ratio[x]*entropies[x]
    return total_entropy

def build_tree(data,features):
    lastcol=[row[-1] for row in data]
    if(len(set(lastcol)))==1:
        node=Node("")
        node.answer=lastcol[0]
        return node
    
    n=len(data[0])-1
    gains=[0]*n
    for col in range(n):
        gains[col]=compute_gain(data,col)
    split=gains.index(max(gains))
    node=Node(features[split])
    fea = features[:split]+features[split+1:]

    
    attr,dic=subtables(data,split,delete=True)
    
    for x in range(len(attr)):
        child=build_tree(dic[attr[x]],fea)
        node.children.append((attr[x],child))
    return node

def print_tree(node,level):
    if node.answer!="":
        print("  "*level,node.answer)
        return
    
    print("  "*level,node.attribute)
    for value,n in node.children:
        print("  "*(level+1),value)
        print_tree(n,level+2)

        
def classify(node,x_test,features):
    if node.answer!="":
        print(node.answer)
        return
    pos=features.index(node.attribute)
    for value, n in node.children:
        if x_test[pos]==value:
            classify(n,x_test,features)
            
'''Main program'''
dataset,features=load_csv("/content/TrainData.csv")
node1=build_tree(dataset,features)

print("The decision tree for the dataset using ID3 algorithm is")
print_tree(node1,0)
testdata,features=load_csv("/content/TestData.csv")

for xtest in testdata:
    print("The test instance:",xtest)
    print("The label for test instance:",end="   ")
    classify(node1,xtest,features)

print("testing on behalf of our real-time data")
ytest=['sunny','mild','normal','weak']
classify(node1,ytest,features)

The decision tree for the dataset using ID3 algorithm is
 Outlook
   rain
     Wind
       strong
         no
       weak
         yes
   overcast
     yes
   sunny
     Humidity
       high
         no
       normal
         yes
The test instance: ['rain', 'cool', 'normal', 'strong']
The label for test instance:   no
The test instance: ['sunny', 'mild', 'normal', 'strong']
The label for test instance:   yes
testing on behalf of our real-time data
yes


In [0]:
import pandas as pd


In [31]:
df=pd.read_csv("/content/salaries.csv")
df

,company,job,degree,salary_more_then_100k
0,google,sales executive,bachelors,0
1,google,sales executive,masters,0
2,google,business manager,bachelors,1
3,google,business manager,masters,1
4,google,computer programmer,bachelors,0
5,google,computer programmer,masters,1
6,abc pharma,sales executive,masters,0
7,abc pharma,computer programmer,bachelors,0
8,abc pharma,business manager,bachelors,0
9,abc pharma,business manager,masters,1


In [0]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
features=df.drop("salary_more_then_100k",axis='columns')
features

,company,job,degree
0,google,sales executive,bachelors
1,google,sales executive,masters
2,google,business manager,bachelors
3,google,business manager,masters
4,google,computer programmer,bachelors
5,google,computer programmer,masters
6,abc pharma,sales executive,masters
7,abc pharma,computer programmer,bachelors
8,abc pharma,business manager,bachelors
9,abc pharma,business manager,masters


In [34]:
labels=df['salary_more_then_100k']
labels

0     0
1     0
2     1
3     1
4     0
5     1
6     0
7     0
8     0
9     1
10    1
11    1
12    1
13    1
14    1
15    1
Name: salary_more_then_100k, dtype: int64

In [0]:
# sring label convering to  numbers[  int/float  ]
from sklearn.preprocessing import LabelEncoder

In [0]:
num_labeler=LabelEncoder()#this is for company,job,degree  number-labelling

In [0]:
features.iloc[:,0]=num_labeler.fit_transform(features.iloc[:,0])#for numeric labelling of the company-coulumn
features.iloc[:,1]=num_labeler.fit_transform(features.iloc[:,1])#similarly for job coulumn
features.iloc[:,2]=num_labeler.fit_transform(features.iloc[:,2])#similarly for degree coulimn

In [37]:
features#for checking 

,company,job,degree
0,2,2,0
1,2,2,1
2,2,0,0
3,2,0,1
4,2,1,0
5,2,1,1
6,0,2,1
7,0,1,0
8,0,0,0
9,0,0,1


In [0]:
#splitting datasets into train & test for further evaluation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state = 47, test_size = 0.20)

In [0]:
#calling decisiontree classifier
#Since, this is a classification problem, we will import the DecisionTreeClassifier function from the sklearn library. 
#Next, we will set the 'criterion' to 'entropy', 
#which sets the measure for splitting the attribute to information gain.
clf=DecisionTreeClassifier(criterion='entropy')

In [0]:
trained=clf.fit(X_train,y_train)#finally feed data

In [49]:
print(X_test)
#shows
"""
    #company        #job                 #degree
    google    computer programmer      bachelors
    facebook  computer programmer      bachelors
    google    bussiness manager        bachelors
    facebook  sales executive          masters
    """

    company  job  degree
4         2    1       0
14        1    1       0
2         2    0       0
11        1    2       1


'\n    #company        #job                 #degree\n    google    computer programmer      bachelors\n    facebook  computer programmer      bachelors\n    google    bussiness manager        bachelors\n    facebook  sales executive          masters\n    '

In [51]:
#now predict with test_features
predicted_100k=trained.predict( X_test)
predicted_100k   #algo-answer

array([1, 1, 1, 0])

In [57]:
#evaluation of decision-trained model
from sklearn.metrics import accuracy_score

#find accuracy score
print('Accuracy Score on train data: ' ,accuracy_score(y_test,predicted_100k))
print('Accuracy Score on train data: ', accuracy_score(y_true=y_train, y_pred=clf.predict(X_train)))


Accuracy Score on train data:  0.5
Accuracy Score on train data:  1.0


In [0]:
#conclusion
#above model is high bias and low varience
"""If our model is too simple and has very few parameters 
then it may have high bias and low variance. 
On the other hand if our model has large number of parameters 
then it’s going to have high variance and low bias. 
So we need to find the right/good balance without overfitting and underfitting the data."""